In [1]:
import numpy as np

In [174]:
K = 5 #K, total sen&com target number

In [207]:
a1_temp = np.array([5.1940, 5.6421, 6.3067, 7.3235, 7.3418], dtype=np.float128)
a1 = a1_temp*10000
a2 = np.array([2.2633, 2.6584, 2.6519, 2.4226, 2.5756], dtype=np.float128)
a3 = np.array([0.9159, 0.9425, 0.9894, 0.9777, 0.9935], dtype=np.float128)
g_temp = np.array([0.6592, 0.0454, 0.8506, 0.9347, 0.6819], dtype=np.float128)
g = g_temp * 0.0001
#情况1：通信0.5W，传感1W.
P_c = 0.5 #power of com (W)
P_s = 1 #sensing power, about 34dBm, working frequcy 77Hz

In [106]:
print(a1,a2,a3)

[51940. 56421. 63067. 73235. 73418.] [2.2633 2.6584 2.6519 2.4226 2.5756] [0.9159 0.9425 0.9894 0.9777 0.9935]


In [208]:
T_m= 3  #total time buget (s)
T_0 = 6e-5   #T_0 = 0.00006  #a slot of each sensing cycles time (s), note that T_0=T_s
E_m = 2.5  #total power buget (J)
#T_m/T_0  #thoeriacl max sensing cycles
c = np.zeros((1,K), dtype=np.float128) + 200  #initial sensing cycle as 200, the accuracy is 0.7911
c_1= np.zeros((1,K), dtype=np.float128) #c_1 denote c + 1
M = (T_m - E_m/P_c)/((1-P_s/P_c)*T_0) #M =  #M denote the mid cycles numbers
y_max_1 = np.floor(T_m/T_0).astype(np.int64)
y_max_2 = np.floor(E_m/(T_0*P_s)).astype(np.int64)
B = 10e+6
# g = 1e-5 
delta = 1e-10

out = np.zeros((1,np.floor(T_m/T_0).astype(np.int64)))
x = np.zeros((np.floor(T_m/T_0).astype(np.int64),K)) #x is 
ind_k= np.zeros((1,np.floor(T_m/T_0).astype(np.int64)))
#Theta = a3 - a1.*1./((c).^a2)

In [209]:
def f_Theta(a1,a2,a3,c):
    Theta = a3 - a1*1/(np.power(c, a2))
    return Theta

In [210]:
#sensing cycles allocation, y is total cycles
for y in list(range(200*K+1, min(y_max_1,y_max_2)+1)): #y的取值是因为每个目标都至少分配了200轮
    #y_1 = y - 200*K  #y_1 is index of 'out'
    if (1-P_s/P_c)*T_0 > 0: #判断正负，如果正执行后者E_m/P_c-P_s/P_c*T_0*y
        if y <= M:  
            c_1 = c + 1  #c_1
            Theta_div = f_Theta(a1,a2,a3,c_1) - f_Theta(a1,a2,a3,c)   #denote the "gradient" of Theta
            max_ind = np.argmax(Theta_div, axis=-1)  #find k^
            max_val = Theta_div[:,max_ind]
            c[:,max_ind] = c[:,max_ind] + 1 
            out[:,y] = np.sum(np.log(f_Theta(a1,a2,a3,c))) +K*np.log(E_m/P_c-P_s/P_c*T_0*y)  # out denote the output value if objective function
        else: #执行前者
            c_1 = c + 1   #
            Theta_div = f_Theta(a1,a2,a3,c_1) - f_Theta(a1,a2,a3,c)   #denote the "gradient" of Theta
            max_ind = np.argmax(Theta_div, axis=-1)  #find k^
            max_val = Theta_div[:,max_ind]
            c[:,max_ind] = c[:,max_ind] + 1 
            out[:,y] = np.sum(np.log(f_Theta(a1,a2,a3,c))) +K*np.log(T_m-T_0*y) 
    else:
        if y <=M: #判断正负，如果负执行前者者T_m-T_0*y
            c_1 = c + 1   #
            Theta_div = f_Theta(a1,a2,a3,c_1) - f_Theta(a1,a2,a3,c)   #denote the "gradient" of Theta
            max_ind = np.argmax(Theta_div, axis=-1)  #find k^
            max_val = Theta_div[:,max_ind]
            c[:,max_ind] = c[:,max_ind] + 1 
            out[:,y] = np.sum(np.log(f_Theta(a1,a2,a3,c))) +K*np.log(T_m-T_0*y) 
        else: #执行后者
            c_1 = c + 1  #c_1
            Theta_div = f_Theta(a1,a2,a3,c_1) - f_Theta(a1,a2,a3,c)   #denote the "gradient" of Theta
            max_ind = np.argmax(Theta_div, axis=-1)  #find k^
            max_val = Theta_div[:,max_ind]
            c[:,max_ind] = c[:,max_ind] + 1 
            out[:,y] = np.sum(np.log(f_Theta(a1,a2,a3,c))) +K*np.log(E_m/P_c-P_s/P_c*T_0*y)  # out denote the output value if objective function
    # if max_ind > 0:
    #     print("max_ind: ", max_ind, "c: ", c[:,max_ind])    
    x[y,max_ind] = c[:,max_ind] 
    ind_k[:,y]= max_ind + 1
    #ind_k(y)= max_ind 

In [211]:
np.sum(ind_k)

112762.0

In [212]:
opt_y_value = np.argmax(out)  #find object y which makes object max
max_out = out[:,opt_y_value]
print(opt_y_value)
opt_c_value = np.max(x[0:opt_y_value,:],axis=0)  #the optimal varialble value (c_1,...c_5)
print(opt_c_value)
#target_index = ind_k(max_y_ind-4:max_y_ind)
#max_c_value
t_test = E_m/P_c-P_s/P_c*T_0*1975
#[max_val, max_ind] = max(Theta_div)  
print(out[:,opt_y_value])

2169
[605. 324. 337. 507. 395.]
[4.99358657]


In [213]:
temp1 = T_m-T_0*opt_y_value
temp2 = E_m/P_c-P_s/P_c*T_0*opt_y_value
t_k = 1/K * min(T_m-T_0*opt_y_value, E_m/P_c-P_s/P_c*T_0*opt_y_value)
opt_rate = np.sum(np.log(f_Theta(a1,a2,a3,opt_c_value)*(t_k/T_m * B *np.log2(1+g*P_c/delta))))

In [215]:
t_k
opt_rate

86.39968693335065665